In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import os
import urllib.request
import re
from datetime import datetime

## 특정 태그(로그인 성공 시 생성) 나올때까지 기다리기
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

def newLoginSession(email):
    '''
    새로운 세션 생성
    - email: DB에 저장된 이메일 입력(이메일 전송용)
    '''
    
    driver = webdriver.Chrome('chromedriver')
    # 창 최대화(최종 구현 시 창 제거 커맨드 입력해야함.)
    # driver.maximize_window()

    driver.get('https://discord.com/login')

    # QR코드가 생성되어 표시될 때 까지 기다림
    # QR_Code_Base64 = driver.find_element_by_class_name('qrCode-2R7t9S').find_element_by_tag_name('img').get_attribute('src')
    QR_Code_Base64 = WebDriverWait(driver, timeout=60).until( EC.presence_of_element_located((By.XPATH, '//*[@id="app-mount"]/div[2]/div/div/div/div/form/div/div/div[3]/div/div/div/div[1]/div[1]/img')) )
    QR_Code_Base64 = QR_Code_Base64.get_attribute('src')
#     print(QR_Code_Base64)
    
    # 메소드 실행
    QR_Code_Send_Email(email, QR_Code_Base64)
    loginCheck(driver)
    crawling(driver, '웨이')

######################################## 이메일 전송 ##########################################
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header
from email.utils import formatdate
from email.mime.base import MIMEBase
import secretAccount as secret

def QR_Code_Send_Email(mailAddr, QR_Code_Base64):
    '''
    디스코드 로그인 QR코드를 메일로 전송하기 위한 코드
    - mailAddr: 수신자의 메일주소
    - QR_Code_Base64: QR코드를 Base64로 변환하여 입력
    '''
    
    # 세션 생성
    s = smtplib.SMTP('smtp.gmail.com', 587)

    # TLS 보안 시작
    s.starttls()

    # 로그인 인증
    s.login(secret.emailID, secret.emailPW) # 비번 포함됨!!!!!! 유출 주의

    # 보낼 메시지 설정
    msg = MIMEMultipart()
    msg['From'] = '디스코드 알리미' # 표시될 이름
    msg['To'] = mailAddr
    msg['Date'] = formatdate(localtime=True)
    msg['Subject'] = Header(s='디스코드 알리미 로그인 QR코드', charset='utf-8')
    # body = MIMEText('첨부된 파일 확인해 주세요.', _charset='utf-8')
    body = MIMEText('디스코드 알리미 로그인 QR코드입니다. <br />' + 
                    '아래의 QR코드를 디스코드 앱 내의 좌측 상단 三 메뉴 -> 우측 하단 디스코드 아이콘 -> <br />' + 
                    '[사용자 설정] 항목의 [QR 코드 스캔] 기능으로 스캔해주세요! <br />' + 
                    '<b>해당 QR코드는 약 1분간 유효합니다.</b><br /><br /><br /><img src=' + QR_Code_Base64 + ' />', 'html')
    msg.attach(body)

    ## 첨부파일 전송: base64 형태로 바로 입력받을 수 있도록 수정함. ## 이거 사용 없이 가능
    part = MIMEBase('application', "octet-stream")
    part.add_header('Content-Transfer-Encoding', 'base64')
    part.add_header('Content-Disposition', 'attachment; filename="QR CODE.png"')
    part.set_payload(QR_Code_Base64.split(sep='data:image/png;base64,')[1]) # 첨부될 base64 기반 문자열 입력
    msg.attach(part)
    
    ## 디버깅용 로그
#     print(part)
#     type(part)

    # msg['Subject'] = '제목 : 메일 보내기 테스트'
    # msg = MIMEText('내용 : 본문내용 테스트', _charset='utf-8')

    # 메일 보내기
    s.sendmail(secret.emailID, mailAddr, msg.as_string())
    # s.send_message(msg) # 이걸로 하면 한글 이름이 제대로 표시되지 않음!

    # 세션 종료
    s.quit()

##################################### 로그인 성공 여부 판단 #######################################

def loginCheck(driver):
    nickname = WebDriverWait(driver, timeout=60).until( EC.presence_of_element_located((By.CLASS_NAME, 'title-338goq')) )
    print('로그인 성공!')
    print('닉네임: ' + nickname.text)

# 닉네임 DB에 기록
# 앱에 ???님 로그인 하신 것을 환영합니다. 띄워주면 좋을듯?

######################################### 크롤링 코드 입력 ########################################

def crawling(driver, keyword):
    time.sleep(4)
    아만URL = "https://discord.com/channels/660684739056762891/724168359171719199" #키워드 선택이랑 서버선택도 해야함 서버선택에서 해당서버 url 들 
    driver.get(아만URL)

    time.sleep(4)
    element = driver.find_elements_by_class_name("contents-2MsGLg")

    check=checking(element)
    while(True):
        context = re.sub(r'\[[^)]*\]', '',element[check].text)
        print(context)
        check=check+1
        if keyword in context:
            print("push")
        else:
            print("no")
        if check == 50:
            break

def checking(element):
    i=0
    check=0
    for _ in element:
        if i>=39:    
            context =element[i].text
            if "떠돌이 상인은" in context:
                check=i
                i=i+1
            else:
                i=i+1
                check=check
        else:
            context = re.sub(r'\[[^)]*\]', '',element[i].text)
            i=i+1
        
    return check        

In [3]:
newLoginSession('positionace@naver.com')




로그인 성공!
닉네임: 대풍


<ipython-input-1-ca8f30ef03e7>:117: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  element = driver.find_elements_by_class_name("contents-2MsGLg")


떠돌이 상인 알림
봇
오늘 오전 11:56
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
✦ 떠돌이 상인은 매 시 30분에 등장하고 55분에 사라집니다.

✦ 키워드 알림 설정은
알림설정 채널에서 할 수 있습니다.

✦ 다음 떠돌이 상인은 아르테미스, 루테란 서부, 루테란 동부 (버트), 토토이크, 아르데타인, 베른 북부, 로헨델, 욘, 파푸니카, 베른 남부 대륙에 10명이 등장합니다. 
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
no
KorLARK
봇
오늘 오후 12:31
 파푸니카 / 티카티카 군락지 / 영웅 호감도 / 알비온 카드 - @영웅 호감도
no
 베른 남부 / 벨리온 유적지 / 킬리언 카드 / 영웅 호감도 - @영웅 호감도
no
 루테란 동부 (버트) / 크로커니스 해변 / 영웅 호감도 / 녹스 카드 - @영웅 호감도
no
